## Лузина Владимира РИС22-2
# Семинар 9. Классификация и кластеризация

Используя имеющиеся статистические данные выполнить следующие шаги:
1) Разметить данные. Если цена уменьшилась или не изменилась и при этом объем продаж
увеличился или не изменился, то присваиваем класс 1 иначе класс 2
2) Предположить, что каждое 3-е значение класса нам не известно
3) Разметить неизвестные значения методом k-NN при k=1 и k=3. Сделать выводы о
правильности определения классов.
4) Нормализовать данные с помощью обезразмеривания
5) Дополнительно разбить данные методом k-nn на кластеры

In [323]:
import pandas as pd
import numpy as np
from typing import List
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


In [324]:
df = pd.DataFrame({'day': [i for i in range(1, 21)],  # дни с 1 по 20
                   'price': [2.61, 5.45, 2.88, 4.28, 2.47, 4.57, 0.15, 0.38, 1.10, 2.43, 1.23, 5.93, 1.44, 1.28, 1.43,
                             1.90, 1.96, 2.08, 3.88, 4.41],  # стоимость
                   'amount': [46, 55, 53, 43, 61, 48, 198, 151, 115, 43, 137, 89, 134, 97, 90, 133, 97., 67, 60,
                              55]})  # число продаж

# df = pdDataFrame({'day': [i for i in range(1, 21)],  # дни с 1 по 20
#                   'price': [3.55, 4.48, 3.27, 4.32, 2.71, 4.49, 0.13, 0.5, 1.16, 3.24, 1.11, 4.98, 1.23, 1.82, 1.35,
#                             1.79, 2.01, 2.18, 3.26, 3.94],  # стоимость
#                   'amount': [50, 55, 59, 45, 60, 50, 210, 160, 140, 50, 143, 100, 130, 92, 114, 112, 97, 83, 66,
#                              60]})  # число продаж

df

,day,price,amount
0,1,2.61,46.0
1,2,5.45,55.0
2,3,2.88,53.0
3,4,4.28,43.0
4,5,2.47,61.0
5,6,4.57,48.0
6,7,0.15,198.0
7,8,0.38,151.0
8,9,1.10,115.0
9,10,2.43,43.0


### Классификация

Признак - класс 1 присваиваем дню, для которого:
  - Цены относительно предыдущего дня уменьшились или не измениись
  - И объём продаж увеличились или не изменился
В противном случае присваиваем класс 2

In [325]:
df["class(F)"] = [0] + [1 if df["price"][i] <= df["price"][i - 1] and df["amount"][i] >= df["amount"][i - 1] else 2 for
                        i in range(1, 20)]
df

,day,price,amount,class(F)
0,1,2.61,46.0,0
1,2,5.45,55.0,2
2,3,2.88,53.0,2
3,4,4.28,43.0,2
4,5,2.47,61.0,1
5,6,4.57,48.0,2
6,7,0.15,198.0,1
7,8,0.38,151.0,2
8,9,1.10,115.0,2
9,10,2.43,43.0,2


In [326]:
distance_mx = np.ndarray(shape=(13, 6))

left_index = 2
two = True
for i in range(len(distance_mx)):
    for j in range(len(distance_mx[0])):
        distance_mx[i][j] = np.linalg.norm(
            df[df["day"] == (j + 1) * 3][["price", "amount"]].values -
            df[df["day"] == left_index][["price", "amount"]].values
        )
    if two:
        left_index += 2
        two = False
    else:
        left_index += 1
        two = True

distance_mx


array([[  3.25651654,   7.05509745,  60.15748083,  34.00338807,
         35.23010644,  12.46422481],
       [ 10.09752445,   5.00840294,  72.07019079,  46.02958288,
         47.08633029,  24.1006224 ],
       [  8.01049936,  13.16852308,  54.01737591,  28.21296865,
         29.01864228,   6.01266164],
       [145.02569738, 150.0651072 ,  83.00543657, 109.15314196,
        108.00758492, 131.0142164 ],
       [ 98.03188257, 103.08518856,  36.00719928,  62.24791161,
         61.00903622,  84.01720062],
       [ 10.01011988,   5.43871308,  72.01228298,  46.13296002,
         47.01063709,  24.00255195],
       [ 84.01620379,  89.06264986,  22.00038409,  48.22955525,
         47.00042553,  70.00516052],
       [ 81.01279899,  86.05693987,  19.00304186,  45.22344635,
         44.00000114,  67.00305665],
       [ 44.0290813 ,  49.1103258 ,  18.00089998,   9.25324267,
          7.00160696,  30.01066477],
       [ 80.00600227,  85.04192437,  18.01776901,  44.18417024,
         43.00256853,  66.0

Классификация при количестве соседей k = 1
выбранный день к ближайшему соседу
сверху дни 3,6,9,12,15,18
слева 2,4,5,7,8,10,11,13,14,16,17,19,20

In [327]:
distance_mx.argmin(axis=0)

array([12,  1,  8, 10,  8,  2], dtype=int64)

Я не смог реализовать это программно, поэтому сделаю такую некрасивую табличку
день | группа  | классификация
  3  |    2    |       2
  6  |    1    |       2
  9  |    2    |       2
  12 |    2    |       2
  15 |    2    |       2
  18 |    2    |       2
  
Значит модель правильно сработало на 5/6, что довольно неплохо, 
но всё равно мало о чём говорит при таком маленьком количестве тестов
Кроме того, почти наверняка такая точность вызвана дисбалансом классов (2-го класса намного больше первого)
но так как это учебная задача оставлю как есть

In [328]:
df

,day,price,amount,class(F)
0,1,2.61,46.0,0
1,2,5.45,55.0,2
2,3,2.88,53.0,2
3,4,4.28,43.0,2
4,5,2.47,61.0,1
5,6,4.57,48.0,2
6,7,0.15,198.0,1
7,8,0.38,151.0,2
8,9,1.10,115.0,2
9,10,2.43,43.0,2


Теперь для трёх значений
верхние индексы на самом деле 3,6,9,12,15
а значения
индекс | тип
   0   |  2
   1   |  2
   2   |  1
   3   |  1
   4   |  2
   5   |  2
   6   |  1
   7   |  1
   8   |  2
   9   |  2
   10  |  2
   11  |  2
   12  |  2

In [329]:
distance_mx.argsort(axis=0)[:3]

array([[12,  1,  8, 10,  8,  2],
       [ 0,  5,  9,  8, 10, 11],
       [11, 12, 10,  2,  2, 12]], dtype=int64)

день | группа  | классификация
  3  |    2    |    2
  6  |    1    |    2
  9  |    2    |    2
  12 |    2    |    2
  15 |    2    |    2
  18 |    2    |    2
Модель опять правильно сработала на 5/6, что, опять же, довольно неплохо, 
но всё равно мало о чём говорит при таком маленьком количестве тестов
Кроме того, почти наверняка такая точность вызвана дисбалансом классов (2-го класса намного больше первого)
но так как это учебная задача ~~и я классифицировал практически вручную~~ оставлю как есть

### Кластеризация

Берём 20 значений из предыдущего задания
для нормализации воспользуемся обезразмериванием (значение делим на максимум)
Цель - получить однородное множество
Порог - среднее по исходной матрице расстояний
Остановка, когда не будет пороговых значений

Нормализация

In [330]:
max_x = df["price"].max()
max_y = df["amount"].max()
df_norm = pd.DataFrame(
    {"day": df["day"], "X": [i / max_x for i in df["price"]], "Y": [i / max_y for i in df["amount"]]})
df_norm

,day,X,Y
0,1,0.440135,0.232323
1,2,0.919056,0.277778
2,3,0.485666,0.267677
3,4,0.721754,0.217172
4,5,0.416526,0.308081
5,6,0.770658,0.242424
6,7,0.025295,1.000000
7,8,0.064081,0.762626
8,9,0.185497,0.580808
9,10,0.409781,0.217172


Создам функцию для построение таблицы расстояний между соседями для всех дней
и ещё одну функцию для добавление нового кластера

In [331]:
def print_distances(df_norm: pd.DataFrame) -> pd.DataFrame:
    distance_mx = np.ndarray(shape=(len(df_norm), len(df_norm)))

    days = df_norm["day"].to_list()

    for i in range(len(days)):
        for j in range(len(days)):
            distance_mx[i][j] = round(np.linalg.norm(
                df_norm[df_norm["day"] == days[i]][["X", "Y"]].values -
                df_norm[df_norm["day"] == days[j]][["X", "Y"]].values
            ), 2)
    return distance_mx


def add_cluster(df_local_norm: pd.DataFrame, name: str, values: List) -> pd.DataFrame:
    # новый кластер
    cluster = df_local_norm[df_local_norm["day"].isin(values)]
    cluster_x = round(cluster["X"].mean(), 2)
    cluster_y = round(cluster["Y"].mean(), 2)

    # удаляет дни кластера из общего датафрейма и добавляем инициализированный кластер
    df_local_norm = df_local_norm[~df_local_norm["day"].isin(values)]
    df_local_norm = pd.concat([df_local_norm, pd.DataFrame({"day": [name], "X": [cluster_x], "Y": [cluster_y]})],
                              ignore_index=True)
    return df_local_norm

Возьму порог как в примере за 0.21
Буду считать это гиперпараметром,
который должен либо подбирается вручную, либо на валидационной выборке

Построю таблицу расстояний между соседями для всех дней

In [332]:
print_distances(df_norm)

array([[0.  , 0.48, 0.06, 0.28, 0.08, 0.33, 0.87, 0.65, 0.43, 0.03, 0.52,
        0.6 , 0.49, 0.34, 0.3 , 0.46, 0.28, 0.14, 0.23, 0.31],
       [0.48, 0.  , 0.43, 0.21, 0.5 , 0.15, 1.15, 0.98, 0.79, 0.51, 0.82,
        0.19, 0.79, 0.73, 0.7 , 0.72, 0.63, 0.57, 0.27, 0.18],
       [0.06, 0.43, 0.  , 0.24, 0.08, 0.29, 0.87, 0.65, 0.43, 0.09, 0.51,
        0.55, 0.48, 0.35, 0.31, 0.44, 0.27, 0.15, 0.17, 0.26],
       [0.28, 0.21, 0.24, 0.  , 0.32, 0.06, 1.05, 0.85, 0.65, 0.31, 0.7 ,
        0.36, 0.66, 0.57, 0.54, 0.61, 0.48, 0.39, 0.11, 0.06],
       [0.08, 0.5 , 0.08, 0.32, 0.  , 0.36, 0.79, 0.58, 0.36, 0.09, 0.44,
        0.6 , 0.41, 0.27, 0.23, 0.38, 0.2 , 0.07, 0.24, 0.33],
       [0.33, 0.15, 0.29, 0.06, 0.36, 0.  , 1.06, 0.88, 0.68, 0.36, 0.72,
        0.31, 0.68, 0.61, 0.57, 0.62, 0.5 , 0.43, 0.13, 0.04],
       [0.87, 1.15, 0.87, 1.05, 0.79, 1.06, 0.  , 0.24, 0.45, 0.87, 0.36,
        1.12, 0.39, 0.54, 0.59, 0.44, 0.59, 0.74, 0.94, 1.02],
       [0.65, 0.98, 0.65, 0.85, 0.58, 0.8

Буду разделять на кластеры вручную. В один кластер попадают пересечения значений меньшие или равные пороговому значению (0.21)
В таблице дни начинаются с 0. Я буду писать в комментариях и коде с 1
В первом кластере будут дни:
1, 3, 5, 10, 18
Во втором кластере будут дни:
2, 4, 6, 12, 20
В третьем:
8, 11, 13
В четвёртом:
9, 14, 15, 16, 17
Значения которые пока без кластера/отдельные кластеры
19
7

Пока что отделю эти 4 кластера и построю новую матрицу расстояний



In [333]:
df_norm_1 = add_cluster(df_norm, "cluster1", [1, 3, 5, 10, 18])
df_norm_1 = add_cluster(df_norm_1, "cluster2", [2, 4, 6, 12, 20])
df_norm_1 = add_cluster(df_norm_1, "cluster3", [8, 11, 13])
df_norm_1 = add_cluster(df_norm_1, "cluster4", [9, 14, 15, 16, 17])
df_norm_1

,day,X,Y
0,7,0.025295,1.00000
1,19,0.654300,0.30303
2,cluster1,0.420000,0.27000
3,cluster2,0.830000,0.29000
4,cluster3,0.170000,0.71000
5,cluster4,0.260000,0.54000


Новая таблица расстояний
где,
0 = 7
1 = 19
2 = cluster1 
3 = cluster2 
4 = cluster3 
5 = cluster4 

In [334]:
print_distances(df_norm_1)

array([[0.  , 0.94, 0.83, 1.07, 0.32, 0.52],
       [0.94, 0.  , 0.24, 0.18, 0.63, 0.46],
       [0.83, 0.24, 0.  , 0.41, 0.51, 0.31],
       [1.07, 0.18, 0.41, 0.  , 0.78, 0.62],
       [0.32, 0.63, 0.51, 0.78, 0.  , 0.19],
       [0.52, 0.46, 0.31, 0.62, 0.19, 0.  ]])

Из этой таблицы видно, что 19-й день можно добавить в кластер 2
И что также объёединить кластеры 3 (8, 11, 13) и кластер (9, 14, 15, 16, 17)
В четвёртом:


In [335]:
df_norm_2 = add_cluster(df_norm, "cluster1", [1, 3, 5, 10, 18])
df_norm_2 = add_cluster(df_norm_2, "cluster2", [2, 4, 6, 12, 19, 20])
df_norm_2 = add_cluster(df_norm_2, "cluster3", [8, 11, 13] + [9, 14, 15, 16, 17])
df_norm_2

,day,X,Y
0,7,0.025295,1.00
1,cluster1,0.420000,0.27
2,cluster2,0.800000,0.29
3,cluster3,0.230000,0.60


Итоговая таблица расстояний выглядит так:
0 = 7
1 = cluster1 
2 = cluster2 
3 = cluster3
Итоговая, так как все значения выше порога

In [336]:
print_distances(df_norm_2)

array([[0.  , 0.83, 1.05, 0.45],
       [0.83, 0.  , 0.38, 0.38],
       [1.05, 0.38, 0.  , 0.65],
       [0.45, 0.38, 0.65, 0.  ]])

Итого получилось разбить датасет на 3 кластера:
первый: дни 1, 3, 5, 10 и 18
второй: дни 2, 4, 6, 12, 19 и 20
третий: дни 8, 9 11, 13, 14, 15, 16 и 17
День 7 можно считать как выбросом, так и отдельным кластером
Данные взяты случайным образом, так что определить это невозможно